In [292]:
#first step is to load the packages that I will need.
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pandas_profiling import ProfileReport
import xlrd


In [293]:
#first step is to load the 2 tabs of data with the revelant info from my Excel Spreadsheet.

xls = pd.ExcelFile('egrid2018_data_v2.xlsx')



df1 = pd.read_excel(xls, 'UNT18')
df2 = pd.read_excel(xls, 'GEN18')

In [294]:
xls.sheet_names  # see all sheet names

['Contents',
 'Idea',
 'UNT18',
 'GEN18',
 'PLNT18',
 'ST18',
 'BA18',
 'SRL18',
 'NRL18',
 'US18',
 'GGL18']

In [295]:
print(df1.columns)

print(df2.columns)

Index(['eGRID 2018 Unit file sequence number', 'Data Year',
       'Plant state abbreviation', 'Plant name',
       'DOE/EIA ORIS plant or facility code', 'Unit ID', 'Prime Mover',
       'Unit Operational Status', 'CAMD Program flag', 'Program code(s)',
       'Unit bottom and firing type', 'Number of associated generators',
       'Unit primary fuel', 'Unit operating hours',
       'Unit unadjusted annual heat input (MMBtu)',
       'Unit unadjusted ozone season heat input (MMBtu)',
       'Unit unadjusted annual NOx emissions (tons)',
       'Unit unadjusted ozone season NOx emissions (tons)',
       'Unit unadjusted annual SO2 emissions (tons)',
       'Unit unadjusted annual CO2 emissions (tons)',
       'Unit unadjusted annual Hg emissions (lbs)',
       'Unit unadjusted annual heat input source',
       'Unit unadjusted ozone season heat input source',
       'Unit unadjusted annual NOx emissions source',
       'Unit unadjusted ozone season NOx emissions source',
       'Unit u

In [296]:
#print(df1.describe())

print(df1['Data Year'].value_counts())

2018    26732
YEAR        1
Name: Data Year, dtype: int64


In [297]:
print(df1.iloc[0,:])
print(df1.head())

eGRID 2018 Unit file sequence number                 SEQUNT18
Data Year                                                YEAR
Plant state abbreviation                             PSTATABB
Plant name                                              PNAME
DOE/EIA ORIS plant or facility code                    ORISPL
Unit ID                                                UNITID
Prime Mover                                             PRMVR
Unit Operational Status                               UNTOPST
CAMD Program flag                                    CAMDFLAG
Program code(s)                                       PRGCODE
Unit bottom and firing type                          BOTFIRTY
Number of associated generators                        NUMGEN
Unit primary fuel                                      FUELU1
Unit operating hours                                    HRSOP
Unit unadjusted annual heat input (MMBtu)               HTIAN
Unit unadjusted ozone season heat input (MMBtu)         HTIOZ
Unit una

In [298]:
# i want to get rid of this extra row up top in both dfs
df1 = df1.drop(df1.index[0])

In [299]:
df2 = df2.drop(df2.index[0])

In [300]:
# I need to combine these datasets, but there is no matching unique identifier I can use to map them.

df1["Plant name"].value_counts()
df1["Plant name"].count()





26732

In [301]:
df2["Plant name"].value_counts()
df2["Plant name"].count()

27944

In [302]:
df1['Plant name'].nunique()

10933

In [303]:
df2['Plant name'].nunique()

10947

In [304]:
#How can i generate a unique value in both tables for each row so that I can merge them on that value?  I can take Plant Name + 1, Plant Name + 2, etc.
#They don't line up correctly and probably aren't meant to be viewed that way, luckily there is another tab for Plant Summary stats that has all of the values together. I'm going to use that. 

df3 = pd.read_excel(xls, 'PLNT18')


In [305]:
print(df3.columns)
print(df3.iloc[0,:])
print(df3.head())

Index(['eGRID2018 Plant file sequence number', 'Data Year',
       'Plant state abbreviation', 'Plant name',
       'DOE/EIA ORIS plant or facility code',
       'Plant transmission or distribution system owner name',
       'Plant transmission or distribution system owner ID', 'Utility name',
       'Utility ID', 'Plant-level sector',
       ...
       'Plant wind generation percent (resource mix)',
       'Plant solar generation percent (resource mix)',
       'Plant geothermal generation percent (resource mix)',
       'Plant other fossil generation percent (resource mix)',
       'Plant other unknown / purchased fuel generation percent (resource mix)',
       'Plant total nonrenewables generation percent (resource mix)',
       'Plant total renewables generation percent (resource mix)',
       'Plant total nonhydro renewables generation percent (resource mix)',
       'Plant total combustion generation percent (resource mix)',
       'Plant total noncombustion generation percent (r

In [306]:
# i want to get rid of this extra row up top in both dfs
df3 = df3.drop(df1.index[0])

In [307]:
print(df3.head())

  eGRID2018 Plant file sequence number Data Year Plant state abbreviation  \
0                             SEQPLT18      YEAR                 PSTATABB   
2                                    2      2018                       AK   
3                                    3      2018                       AK   
4                                    4      2018                       AK   
5                                    5      2018                       AK   

                         Plant name DOE/EIA ORIS plant or facility code  \
0                             PNAME                              ORISPL   
2  Agrium Kenai Nitrogen Operations                               54452   
3                          Alakanuk                               57053   
4               Allison Creek Hydro                               58982   
5                            Ambler                               60243   

  Plant transmission or distribution system owner name  \
0                           

In [308]:
columns = df3.columns.to_list()

In [309]:
print(columns)

#I can get rid of a lot of these columns.



['eGRID2018 Plant file sequence number', 'Data Year', 'Plant state abbreviation', 'Plant name', 'DOE/EIA ORIS plant or facility code', 'Plant transmission or distribution system owner name', 'Plant transmission or distribution system owner ID', 'Utility name', 'Utility ID', 'Plant-level sector', 'Balancing Authority Name', 'Balancing Authority Code', 'NERC region acronym', 'eGRID subregion acronym', 'eGRID subregion name', 'Plant associated ISO/RTO Territory ', 'Plant FIPS state code', 'Plant FIPS county code', 'Plant county name', 'Plant latitude', 'Plant longitude', 'Number of units', 'Number of generators', 'Plant primary fuel', 'Plant primary coal/oil/gas/ other fossil fuel category', 'Flag indicating if the plant burned or generated any amount of coal', 'Plant capacity factor', 'Plant nameplate capacity (MW)', 'Nonbaseload Factor', 'Biogas/ biomass plant adjustment flag', 'Combined heat and power (CHP) plant adjustment flag', 'CHP plant useful thermal output (MMBtu)', 'CHP plant p

In [310]:
# I will just make a new df of the columsn I care about

columns_of_interest = ['Data Year', 'Plant state abbreviation', 'Utility name', 'Plant name', 'Plant-level sector', 'Balancing Authority Name', 'eGRID subregion name', 'Plant latitude', 'Plant longitude', 'Number of units', 'Number of generators', 'Plant primary coal/oil/gas/ other fossil fuel category','Flag indicating if the plant burned or generated any amount of coal', 'Plant capacity factor', 'Plant nameplate capacity (MW)', 'Nonbaseload Factor', 'Plant annual net generation (MWh)', 'Plant annual CO2 emissions (tons)', 'Plant annual CH4 emissions (lbs)', 'Plant annual CO2 total output emission rate (lb/MWh)', 'Plant annual CH4 total output emission rate (lb/MWh)', 'Plant annual total nonrenewables net generation (MWh)', 'Plant annual total renewables net generation (MWh)', 'Plant total nonrenewables generation percent (resource mix)', 'Plant total renewables generation percent (resource mix)']




In [311]:
df3 = df3[columns_of_interest]


print(df3.head())

  Data Year Plant state abbreviation                   Utility name  \
0      YEAR                 PSTATABB                       UTLSRVNM   
2      2018                       AK                  Agrium US Inc   
3      2018                       AK  Alaska Village Elec Coop, Inc   
4      2018                       AK   Copper Valley Elec Assn, Inc   
5      2018                       AK  Alaska Village Elec Coop, Inc   

                         Plant name Plant-level sector  \
0                             PNAME             SECTOR   
2  Agrium Kenai Nitrogen Operations     Industrial CHP   
3                          Alakanuk   Electric Utility   
4               Allison Creek Hydro   Electric Utility   
5                            Ambler   Electric Utility   

  Balancing Authority Name eGRID subregion name Plant latitude  \
0                   BANAME               SRNAME            LAT   
2     Alaska Miscellaneous     ASCC Alaska Grid        60.6732   
3     Alaska Miscellaneous

In [312]:
df = df3.drop(df3.index[0])

In [313]:
print(df.head())

  Data Year Plant state abbreviation                         Utility name  \
2      2018                       AK                        Agrium US Inc   
3      2018                       AK        Alaska Village Elec Coop, Inc   
4      2018                       AK         Copper Valley Elec Assn, Inc   
5      2018                       AK        Alaska Village Elec Coop, Inc   
6      2018                       AK  Anchorage Municipal Light and Power   

                         Plant name Plant-level sector  \
2  Agrium Kenai Nitrogen Operations     Industrial CHP   
3                          Alakanuk   Electric Utility   
4               Allison Creek Hydro   Electric Utility   
5                            Ambler   Electric Utility   
6                       Anchorage 1   Electric Utility   

            Balancing Authority Name eGRID subregion name Plant latitude  \
2               Alaska Miscellaneous     ASCC Alaska Grid        60.6732   
3               Alaska Miscellaneous

In [314]:
df.describe()

,Data Year,Plant state abbreviation,Utility name,Plant name,Plant-level sector,Balancing Authority Name,eGRID subregion name,Plant latitude,Plant longitude,Number of units,...,Nonbaseload Factor,Plant annual net generation (MWh),Plant annual CO2 emissions (tons),Plant annual CH4 emissions (lbs),Plant annual CO2 total output emission rate (lb/MWh),Plant annual CH4 total output emission rate (lb/MWh),Plant annual total nonrenewables net generation (MWh),Plant annual total renewables net generation (MWh),Plant total nonrenewables generation percent (resource mix),Plant total renewables generation percent (resource mix)
count,10963,10963,10963,10963,10955,10963,10963,10935.000,10935.00,10963,...,3351.0,9133.0,3467.0,3502.0,3523.0,3555.0,9143.0,9143.0,8607.0,8607.0
unique,1,51,5019,10932,7,77,26,10583.000,10666.00,37,...,1552.0,7988.0,2957.0,3037.0,2861.0,485.0,2859.0,5534.0,353.0,355.0
top,2018,CA,Cypress Creek Renewables,Jefferson Solar,IPP Non-CHP,Midcontinent Independent Transmission System O...,WECC California,39.436,-76.22,1,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
freq,10963,1613,197,2,6127,1830,1552,7.000,7.00,6027,...,1627.0,351.0,388.0,383.0,618.0,537.0,5962.0,3162.0,5599.0,5597.0


In [315]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10963 entries, 2 to 10964
Data columns (total 25 columns):
 #   Column                                                               Non-Null Count  Dtype 
---  ------                                                               --------------  ----- 
 0   Data Year                                                            10963 non-null  object
 1   Plant state abbreviation                                             10963 non-null  object
 2   Utility name                                                         10963 non-null  object
 3   Plant name                                                           10963 non-null  object
 4   Plant-level sector                                                   10955 non-null  object
 5   Balancing Authority Name                                             10963 non-null  object
 6   eGRID subregion name                                                 10963 non-null  object
 7   Plant latitud

In [316]:
colsnums = df.columns.drop(['Data Year', 'Plant state abbreviation', 'Utility name', 'Plant name', 'Plant-level sector', 'Balancing Authority Name', 'eGRID subregion name', 'Plant primary coal/oil/gas/ other fossil fuel category', 'Flag indicating if the plant burned or generated any amount of coal'])

df[colsnums] = df[colsnums].apply(pd.to_numeric, errors='coerce')

In [317]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10963 entries, 2 to 10964
Data columns (total 25 columns):
 #   Column                                                               Non-Null Count  Dtype  
---  ------                                                               --------------  -----  
 0   Data Year                                                            10963 non-null  object 
 1   Plant state abbreviation                                             10963 non-null  object 
 2   Utility name                                                         10963 non-null  object 
 3   Plant name                                                           10963 non-null  object 
 4   Plant-level sector                                                   10955 non-null  object 
 5   Balancing Authority Name                                             10963 non-null  object 
 6   eGRID subregion name                                                 10963 non-null  object 
 7   Plan

In [318]:
print(df.describe(include='all'))

        Data Year Plant state abbreviation              Utility name  \
count     10963.0                    10963                     10963   
unique        1.0                       51                      5019   
top        2018.0                       CA  Cypress Creek Renewables   
freq      10963.0                     1613                       197   
mean          NaN                      NaN                       NaN   
std           NaN                      NaN                       NaN   
min           NaN                      NaN                       NaN   
25%           NaN                      NaN                       NaN   
50%           NaN                      NaN                       NaN   
75%           NaN                      NaN                       NaN   
max           NaN                      NaN                       NaN   

             Plant name Plant-level sector  \
count             10963              10955   
unique            10932                  7 

In [319]:
nas=pd.DataFrame(df.isnull().sum().sort_values(ascending=False)/len(df),columns = ['percent'])
pos = nas['percent'] > 0
nas[pos]

,percent
Flag indicating if the plant burned or generated any amount of coal,0.964426
Nonbaseload Factor,0.694335
Plant annual CO2 emissions (tons),0.683754
Plant annual CH4 emissions (lbs),0.680562
Plant annual CO2 total output emission rate (lb/MWh),0.678646
Plant annual CH4 total output emission rate (lb/MWh),0.675727
Plant total nonrenewables generation percent (resource mix),0.214905
Plant total renewables generation percent (resource mix),0.214905
Plant annual net generation (MWh),0.166925
Plant capacity factor,0.166925


In [320]:
df['Plant primary coal/oil/gas/ other fossil fuel category'].value_counts()

SOLAR         3491
GAS           2003
HYDRO         1541
WIND          1340
OIL            943
BIOMASS        797
COAL           408
OTHF           192
GEOTHERMAL      79
NUCLEAR         65
OFSL            25
Name: Plant primary coal/oil/gas/ other fossil fuel category, dtype: int64

In [321]:
#I only want the columns that have a MWh metric because that means that they generated power in 2018.   

df = df[df['Plant annual net generation (MWh)'].notna()]

In [322]:

df['Flag indicating if the plant burned or generated any amount of coal'].fillna('No', inplace=True)

In [323]:
df['Flag indicating if the plant burned or generated any amount of coal'].value_counts()

No     8743
Yes     390
Name: Flag indicating if the plant burned or generated any amount of coal, dtype: int64

In [324]:
di = {"Yes": 1, "No": 2}

df = df.replace({"Flag indicating if the plant burned or generated any amount of coal": di})

In [325]:
df['Flag indicating if the plant burned or generated any amount of coal'].value_counts()

2    8743
1     390
Name: Flag indicating if the plant burned or generated any amount of coal, dtype: int64

In [326]:
df['Plant annual CO2 emissions (tons)'].fillna(0, inplace=True)
df['Plant annual CH4 emissions (lbs)'].fillna(0, inplace=True)
df['Plant annual CO2 total output emission rate (lb/MWh)'].fillna(0, inplace=True)
df['Plant annual CH4 total output emission rate (lb/MWh)'].fillna(0, inplace=True)
df['Plant total nonrenewables generation percent (resource mix)'].fillna(0, inplace=True)
df['Plant total renewables generation percent (resource mix)'].fillna(0, inplace=True)
df['Nonbaseload Factor'].fillna(0, inplace=True)


In [327]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9133 entries, 3 to 10964
Data columns (total 25 columns):
 #   Column                                                               Non-Null Count  Dtype  
---  ------                                                               --------------  -----  
 0   Data Year                                                            9133 non-null   object 
 1   Plant state abbreviation                                             9133 non-null   object 
 2   Utility name                                                         9133 non-null   object 
 3   Plant name                                                           9133 non-null   object 
 4   Plant-level sector                                                   9125 non-null   object 
 5   Balancing Authority Name                                             9133 non-null   object 
 6   eGRID subregion name                                                 9133 non-null   object 
 7   Plant

In [328]:
df['Plant primary coal/oil/gas/ other fossil fuel category'].value_counts()

SOLAR         2719
GAS           1715
HYDRO         1448
WIND          1103
OIL            820
BIOMASS        710
COAL           345
OTHF           127
GEOTHERMAL      64
NUCLEAR         61
OFSL            19
Name: Plant primary coal/oil/gas/ other fossil fuel category, dtype: int64

In [329]:
df = df[df['Plant-level sector'].notna()]

df = df[df['Plant primary coal/oil/gas/ other fossil fuel category'].notna()]


In [330]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9123 entries, 3 to 10964
Data columns (total 25 columns):
 #   Column                                                               Non-Null Count  Dtype  
---  ------                                                               --------------  -----  
 0   Data Year                                                            9123 non-null   object 
 1   Plant state abbreviation                                             9123 non-null   object 
 2   Utility name                                                         9123 non-null   object 
 3   Plant name                                                           9123 non-null   object 
 4   Plant-level sector                                                   9123 non-null   object 
 5   Balancing Authority Name                                             9123 non-null   object 
 6   eGRID subregion name                                                 9123 non-null   object 
 7   Plant

In [331]:
df.describe()

,Plant latitude,Plant longitude,Number of units,Number of generators,Flag indicating if the plant burned or generated any amount of coal,Plant capacity factor,Plant nameplate capacity (MW),Nonbaseload Factor,Plant annual net generation (MWh),Plant annual CO2 emissions (tons),Plant annual CH4 emissions (lbs),Plant annual CO2 total output emission rate (lb/MWh),Plant annual CH4 total output emission rate (lb/MWh),Plant annual total nonrenewables net generation (MWh),Plant annual total renewables net generation (MWh),Plant total nonrenewables generation percent (resource mix),Plant total renewables generation percent (resource mix)
count,9123.000000,9123.000000,9123.000000,9123.000000,9123.000000,9123.000000,9123.000000,9123.000000,9.123000e+03,9.123000e+03,9.123000e+03,9123.000000,9123.000000,9.123000e+03,9.123000e+03,9123.000000,9123.000000
mean,39.091771,-94.766196,2.537652,2.669188,1.957361,0.265502,146.314074,0.264082,4.558139e+05,2.158941e+05,3.872766e+04,535.747717,0.043284,3.782694e+05,7.754396e+04,0.299915,0.642645
std,5.535123,18.765595,3.034549,3.188165,0.202054,0.225919,408.742999,0.406613,1.779943e+06,1.134127e+06,2.473684e+05,2122.169629,0.173005,1.746377e+06,4.193781e+05,0.454330,0.475502
min,18.974200,-171.712439,1.000000,1.000000,1.000000,0.000000,1.000000,0.000000,-2.313523e+06,0.000000e+00,0.000000e+00,0.000000,0.000000,-2.313523e+06,-9.826220e+05,0.000000,0.000000
25%,35.217378,-111.787366,1.000000,1.000000,2.000000,0.091615,3.000000,0.000000,2.785000e+03,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000
50%,39.560833,-91.542500,1.000000,1.000000,2.000000,0.215180,10.000000,0.000000,1.348500e+04,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000e+00,3.660000e+03,0.000000,1.000000
75%,42.537125,-78.691175,3.000000,3.000000,2.000000,0.386075,77.600000,0.572402,1.388615e+05,6.098985e+02,9.677950e+01,735.406000,0.017000,1.125047e+03,2.867100e+04,1.000000,1.000000
max,71.292000,-67.003300,73.000000,73.000000,2.000000,1.693300,6809.000000,1.000000,3.109726e+07,2.015573e+07,4.594840e+06,121798.400000,3.377000,3.109726e+07,2.104256e+07,1.000000,1.000000


In [332]:
dfcats= df.groupby('Plant primary coal/oil/gas/ other fossil fuel category').mean()
print(dfcats)

                                                    Plant latitude  \
Plant primary coal/oil/gas/ other fossil fuel c...                   
BIOMASS                                                  38.639380   
COAL                                                     39.370142   
GAS                                                      37.736257   
GEOTHERMAL                                               37.122777   
HYDRO                                                    41.172455   
NUCLEAR                                                  38.145907   
OFSL                                                     37.479855   
OIL                                                      41.980097   
OTHF                                                     37.451560   
SOLAR                                                    37.865036   
WIND                                                     39.925992   

                                                    Plant longitude  \
Plant primary coal

In [333]:
df.nunique()

Data Year                                                                 1
Plant state abbreviation                                                 51
Utility name                                                           4318
Plant name                                                             9100
Plant-level sector                                                        7
Balancing Authority Name                                                 76
eGRID subregion name                                                     26
Plant latitude                                                         8871
Plant longitude                                                        8932
Number of units                                                          36
Number of generators                                                     39
Plant primary coal/oil/gas/ other fossil fuel category                   11
Flag indicating if the plant burned or generated any amount of coal       2
Plant capaci

In [334]:
pd.concat(g for _, g in df.groupby("Plant name") if len(g) > 1)

,Data Year,Plant state abbreviation,Utility name,Plant name,Plant-level sector,Balancing Authority Name,eGRID subregion name,Plant latitude,Plant longitude,Number of units,...,Nonbaseload Factor,Plant annual net generation (MWh),Plant annual CO2 emissions (tons),Plant annual CH4 emissions (lbs),Plant annual CO2 total output emission rate (lb/MWh),Plant annual CH4 total output emission rate (lb/MWh),Plant annual total nonrenewables net generation (MWh),Plant annual total renewables net generation (MWh),Plant total nonrenewables generation percent (resource mix),Plant total renewables generation percent (resource mix)
8218,2018,OH,City of Woodsfield - (OH),Anadarko,Electric Utility,"PJM Interconnection, LLC",RFC West,39.756982,-81.115503,6,...,1.000000,3.400000e+01,2.646000e+01,2.143,1556.471,0.063,3.400000e+01,0.000,1.000000,0.000000
8422,2018,OK,"Western Farmers Elec Coop, Inc",Anadarko,Electric Utility,Southwest Power Pool,SPP South,35.084700,-98.230000,9,...,0.786148,1.538376e+06,8.396840e+05,31300.477,1091.650,0.020,1.538376e+06,0.000,1.000000,0.000000
587,2018,CA,Ecos Energy LLC,Bear Creek Solar,IPP Non-CHP,California Independent System Operator,WECC California,38.136389,-121.148889,1,...,0.000000,3.367000e+03,0.000000e+00,0.000,0.000,0.000,0.000000e+00,3367.000,0.000000,1.000000
6032,2018,NC,Cypress Creek Renewables,Bear Creek Solar,IPP Non-CHP,Duke Energy Progress East,SERC Virginia/Carolina,35.540344,-79.449053,1,...,0.000000,3.918000e+03,0.000000e+00,0.000,0.000,0.000,0.000000e+00,3918.000,0.000000,1.000000
244,2018,AR,USCE-Little Rock District,Beaver Dam,Electric Utility,Southwestern Power Administration,SPP South,36.421507,-93.847373,2,...,0.000000,1.193800e+05,0.000000e+00,0.000,0.000,0.000,0.000000e+00,119380.000,0.000000,1.000000
8779,2018,PA,Beaver Dam Energy LLC,Beaver Dam,IPP Non-CHP,"PJM Interconnection, LLC",RFC East,41.611389,-76.843056,5,...,0.774491,6.462000e+04,2.931215e+04,1105.703,907.216,0.017,6.462000e+04,0.000,1.000000,0.000000
3305,2018,ID,Idaho Power Co,Cascade Dam,Electric Utility,Idaho Power Company,WECC Northwest,44.525600,-116.048300,2,...,0.000000,4.687900e+04,0.000000e+00,0.000,0.000,0.000,0.000000e+00,46879.000,0.000000,1.000000
4934,2018,MI,"STS Hydropower, LLC",Cascade Dam,IPP Non-CHP,Midcontinent Independent Transmission System O...,RFC Michigan,42.909408,-85.498880,1,...,0.000000,8.104000e+03,0.000000e+00,0.000,0.000,0.000,0.000000e+00,8104.000,0.000000,1.000000
749,2018,CA,Coalinga Cogeneration Co,Coalinga Cogeneration Facility,IPP CHP,California Independent System Operator,WECC California,36.180700,-120.388500,1,...,0.000000,0.000000e+00,0.000000e+00,0.000,0.000,0.000,0.000000e+00,0.000,0.000000,0.000000
750,2018,CA,Aera Energy LLC-Coalinga,Coalinga Cogeneration Facility,Industrial CHP,California Independent System Operator,WECC California,36.170317,-120.364084,2,...,0.000000,4.867200e+04,1.544277e+04,582.527,634.565,0.012,4.867200e+04,0.000,1.000000,0.000000


In [335]:
print(df.head())

  Data Year Plant state abbreviation                         Utility name  \
3      2018                       AK        Alaska Village Elec Coop, Inc   
4      2018                       AK         Copper Valley Elec Assn, Inc   
5      2018                       AK        Alaska Village Elec Coop, Inc   
6      2018                       AK  Anchorage Municipal Light and Power   
7      2018                       AK        Inside Passage Elec Coop, Inc   

            Plant name Plant-level sector           Balancing Authority Name  \
3             Alakanuk   Electric Utility               Alaska Miscellaneous   
4  Allison Creek Hydro   Electric Utility               Alaska Miscellaneous   
5               Ambler   Electric Utility               Alaska Miscellaneous   
6          Anchorage 1   Electric Utility  Anchorage Municipal Light & Power   
7               Angoon   Electric Utility               Alaska Miscellaneous   

  eGRID subregion name  Plant latitude  Plant longitude 